# Transformer based model for German Language Summarisation Task

Context of rthe notebook

## Requirements and Dependencies

In [ ]:
! pip install datasets evaluate transformers rouge-score nltk transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=459669ad10094d49578a9854770bf38450924521c494e80e49e077525b7e436e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff

In [ ]:
!apt install git-lfs

In [ ]:
# HuggingFace Transformers
import transformers
from transformers.utils import send_example_telemetry
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from huggingface_hub import notebook_login

import numpy as np

# Natural Language Toolkit
import nltk

In [ ]:
notebook_login()

In [ ]:
nltk.download('punkt')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:

print(transformers.__version__)

4.37.2


In [ ]:
send_example_telemetry("summarization_notebook", framework="pytorch")

## Fine-tuning T5 Model on German T5 data

In [ ]:
model_checkpoint = "t5-small"

## Loading the dataset

In [ ]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("GEM/mlsum", "de")
metric = load("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/220748 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11392 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10695 [00:00<?, ? examples/s]

Generating challenge_train_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_validation_sample split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating challenge_test_covid split:   0%|          | 0/5058 [00:00<?, ? examples/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
print("Dataset")
print("gem_id:", raw_datasets['train']['gem_id'][0])
print("text:", raw_datasets['train']['text'][0])
print("target:", raw_datasets['train']['target'][0])

Dataset
gem_id: mlsum_de-train-0
text: Transport im Viehwaggon, Fleischgeruch in der Luft, Selektion mit Doktor Mengele: Holocaust-Überlebende Lisa Miková erinnert sich an das Vernichtungslager Auschwitz-Birkenau. Lisa Miková kam 1922 in Prag zur Welt. In ihrer weitgehend säkularen jüdischen Familie wurde Deutsch und Tschechisch gesprochen. 1942 wurde sie ins KZ Theresienstadt deportiert. Von dort wurden zunächst ihre Eltern, dann, im Herbst 1944, ihr Ehemann František ins Vernichtungslager Auschwitz geschafft. Lisa Miková meldete sich kurz später freiwillig für einen Frauentransport. Detailansicht öffnen Lisa Miková im Jahre 1939. Damals erlebte sie, wie die Wehrmacht in Prag einmarschierte. Die Deutschen enteigneten sofort das Geschäft ihres Vaters. (Foto: oh) Welche Qualen Lisa Miková durchlebte und wie sie überlebte, schildert sie immer wieder tschechischen und deutschen Schülern. Die agile Seniorin engagiert sich unter anderem in der Stiftung Brücke/Most, die sich für die deutsch-

In [ ]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

## Preprocessing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:1])

{'input_ids': [[21603, 10, 7608, 256, 1813, 15, 107, 15238, 5307, 6, 20045, 1304, 2295, 16, 74, 10012, 6, 180, 400, 12696, 181, 531, 10377, 13327, 109, 10, 26178, 18, 14858, 1152, 8766, 221, 11712, 2133, 9789, 2975, 26938, 289, 46, 211, 781, 21646, 425, 7, 13298, 1392, 524, 15686, 18, 279, 12546, 35, 402, 5, 11712, 2133, 9789, 2975, 6511, 957, 2884, 16, 6110, 122, 881, 3779, 5, 86, 1159, 52, 9903, 8615, 26, 3, 7, 1864, 157, 4885, 35, 3, 354, 1272, 26, 1779, 7453, 1177, 3, 18609, 64, 332, 3992, 1436, 860, 3, 18007, 5, 24466, 1177, 680, 16, 7, 480, 956, 290, 7, 23, 35, 6208, 20, 1493, 1378, 5, 6560, 5048, 3163, 170, 10151, 1159, 9971, 6, 1352, 6, 256, 21073, 21930, 6, 1650, 262, 88, 2434, 8177, 17, 23, 2, 15, 157, 16, 7, 781, 21646, 425, 7, 13298, 1392, 524, 15686, 3, 27551, 5, 11712, 2133, 9789, 2975, 3, 10114, 15, 17, 15, 289, 7755, 8908, 6277, 20494, 218, 595, 8608, 26803, 5, 20760, 3247, 362, 17, 27317, 11712, 2133, 9789, 2975, 256, 3861, 957, 3288, 5, 878, 11014, 25961, 15, 680, 6, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/220748 [00:00<?, ? examples/s]

Map:   0%|          | 0/11392 [00:00<?, ? examples/s]

Map:   0%|          | 0/10695 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5058 [00:00<?, ? examples/s]

## Traning a Seq-2-Seq model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
batch_size = 32
epochs = 3
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"i",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.998700,1.718372,25.635200,17.036400,23.763500,24.406500,18.990300
2,0.962400,1.699594,25.813200,17.173200,23.913100,24.574400,18.988500


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1707279536.819c6eb87b51.2124.5:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mach-12/t5-small-finetuned-mlsum-de/commit/243345c342a984ea2bc1a1e56823d41b86bf2bf3', commit_message='End of training', commit_description='', oid='243345c342a984ea2bc1a1e56823d41b86bf2bf3', pr_url=None, pr_revision=None, pr_num=None)

## Prediction code

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("mach-12/t5-small-finetuned-mlsum-de")

# Tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained("mach-12/t5-small-finetuned-mlsum-de")

# Summary pair
test_sentence = ["In einem kleinen Dorf, versteckt zwischen sanften Hügeln und grünen Wäldern, lebte Anna, eine junge Frau mit einem großen Traum. Schon seit ihrer Kindheit faszinierte sie die Welt der Kräuter und deren heilende Kräfte. Während andere Kinder spielten, verbrachte sie ihre Zeit damit, die Flora der Umgebung zu erkunden und von ihrer Großmutter, einer erfahrenen Heilerin, zu lernen. Annas Wunsch war es, ihr Wissen weiterzugeben und Menschen mit der Kraft der Natur zu helfen."]
test_summarization = ["Anna, eine junge Frau aus einem kleinen Dorf, träumt davon, Heilerin zu werden. Schon als Kind lernte sie von ihrer Großmutter über die heilenden Kräfte von Kräutern und möchte ihr Wissen nutzen, um anderen Menschen zu helfen."]

# Prompt
input_prompt = f"summarise: {test_sentence[0]}"

# Use the tokenizer to encode the input text
input_ids = tokenizer(input_prompt, return_tensors="pt")["input_ids"]

# Generate predictions using the model
output = model.generate(
    input_ids=input_ids,
    max_length=65,           # Maximum length of generated text
    num_beams=4,             # Number of beams for beam search
    no_repeat_ngram_size=2,  # Avoid repeating n-grams in the generated text
)



# Decode the generated text tokens back to human-readable text
predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Compute metric
rouge_score = metric.compute(predictions=[predicted_text], references=test_summarization)

# Display Output
print("Rouge Score:")
print(rouge_score)
print()
print(f"Input: \n {test_sentence[0]}\n" )
print(f"Predicted summary: \n{predicted_text}\n")
print(f"Actual summary: \n{test_summarization[0]}")


Rouge Score:
{'rouge1': 0.37209302325581395, 'rouge2': 0.09523809523809525, 'rougeL': 0.2558139534883721, 'rougeLsum': 0.2558139534883721}

Input: 
 In einem kleinen Dorf, versteckt zwischen sanften Hügeln und grünen Wäldern, lebte Anna, eine junge Frau mit einem großen Traum. Schon seit ihrer Kindheit faszinierte sie die Welt der Kräuter und deren heilende Kräfte. Während andere Kinder spielten, verbrachte sie ihre Zeit damit, die Flora der Umgebung zu erkunden und von ihrer Großmutter, einer erfahrenen Heilerin, zu lernen. Annas Wunsch war es, ihr Wissen weiterzugeben und Menschen mit der Kraft der Natur zu helfen.

Predicted summary: 
Anna, eine junge Frau mit einem großen Traum. Schon seit ihrer Kindheit faszinierte sie die Welt der Kräuter und deren heilende Kräfte. Während andere Kinder spielten, verbrachte sie ihre Zeit damit, die Flora der Umgebung zu erkunden und von ihre

Actual summary: 
Anna, eine junge Frau aus einem kleinen Dorf, träumt davon, Heilerin zu werden. Schon al